In [2]:
from model import HelicobacterClassifier
import torch

from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch

for n in range(5):
    model = HelicobacterClassifier()

    state_dict = torch.load(
        rf"HelicobacterClassifier_fold{n}.pth")
    model.load_state_dict(state_dict)
    globals()[f'model_fold_{n}'] = model

# Load models into a list
models = [globals()[f'model_fold_{n}'] for n in range(5)]

/var/folders/7s/p9vc6dt51918j4q47v387x880000gn/T/ipykernel_48303/3108494176.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [4]:
import pandas as pd 
path_to_annotated = r'HP_WSI-CoordAnnotatedPatches.xlsx'
path_to_Patient_Diagnois = r'PatientDiagnosis.csv'
patient_diagnosisDF = pd.read_csv(path_to_Patient_Diagnois)
annotated_patchesDF = pd.read_excel(path_to_annotated) 

print(patient_diagnosisDF.head())
print(annotated_patchesDF.head())

patient_diagnosisDF = patient_diagnosisDF[(patient_diagnosisDF['DENSITAT'] == 'ALTA') | (patient_diagnosisDF['DENSITAT'] == 'NEGATIVA')]
annotated_patchesDF = annotated_patchesDF[annotated_patchesDF['Presence'] != 0]


# Group by patient_id and count the number of positive and negative patches
grouped = annotated_patchesDF.groupby(['Pat_ID'])
grouped = annotated_patchesDF.groupby('Pat_ID').agg(
    number_of_positive_patches=('Presence', lambda x: (x == 1).sum()),
    number_of_negative_patches=('Presence', lambda x: (x == -1).sum())
).reset_index()

# Inlcude in gropued pateint_diagnosisDF['DENSITAT'] based on the id
grouped.head()
grouped = grouped.merge(patient_diagnosisDF, left_on='Pat_ID', right_on='CODI', how='inner')
grouped.head()
grouped = grouped.drop(columns=['CODI'])

# OBJECTIVE  have a dataframe with the following columns: patient_id, number_of_positive_patches, number_of_negative_patches, diagnosis, prediction
print(patient_diagnosisDF)

densitat = patient_diagnosisDF [patient_diagnosisDF['CODI'] == "B22-03"]
print(list(densitat['DENSITAT']))

     CODI  DENSITAT
0  B22-01     BAIXA
1  B22-02     BAIXA
2  B22-03  NEGATIVA
3  B22-04  NEGATIVA
4  B22-05  NEGATIVA
    Pat_ID  Section_ID  Window_ID      i      j    h    w  Presence
0  B22-129           0        659   7477  11978  256  256        -1
1   B22-68           0        131   6597  12009  256  256        -1
2   B22-68           0        141   5100  10737  256  256        -1
3   B22-68           0        290   5015  14908  256  256        -1
4   B22-68           0        298  11626  13928  256  256        -1
        CODI  DENSITAT
2     B22-03  NEGATIVA
3     B22-04  NEGATIVA
4     B22-05  NEGATIVA
5     B22-06  NEGATIVA
6     B22-07  NEGATIVA
..       ...       ...
304  B22-311      ALTA
305  B22-312      ALTA
306  B22-313      ALTA
307  B22-314  NEGATIVA
308  B22-315  NEGATIVA

[237 rows x 2 columns]
['NEGATIVA']


In [5]:
import pathlib 
grouped

# Create a pathlib object to store the results

path_to_holdout =  r"HoldOut"
# iterate over the directories in holdout  and include the name of the directory in a list
holdout_directories = [x for x in pathlib.Path(path_to_holdout).iterdir() if x.is_dir()]

# Extract the name of the directories without the full path
holdout_directories = [x.name[:-2] for x in holdout_directories]

holdout_directories

['B22-161',
 'B22-88',
 'B22-04',
 'B22-82',
 'B22-286',
 'B22-44',
 'B22-65',
 'B22-261',
 'B22-225',
 'B22-100',
 'B22-282',
 'B22-314',
 'B22-271',
 'B22-75',
 'B22-196',
 'B22-135',
 'B22-310',
 'B22-14',
 'B22-231',
 'B22-31',
 'B22-10',
 'B22-198',
 'B22-96',
 'B22-207',
 'B22-226',
 'B22-247',
 'B22-266',
 'B22-85',
 'B22-128',
 'B22-222',
 'B22-203',
 'B22-81',
 'B22-66',
 'B22-281',
 'B22-62',
 'B22-09',
 'B22-03',
 'B22-49',
 'B22-309',
 'B22-209',
 'B22-262',
 'B22-243',
 'B22-285',
 'B22-268',
 'B22-07',
 'B22-236',
 'B22-78',
 'B22-238',
 'B22-272',
 'B22-295',
 'B22-132',
 'B22-36',
 'B22-213',
 'B22-13',
 'B22-32',
 'B22-72',
 'B22-257',
 'B22-19',
 'B22-136',
 'B22-259',
 'B22-17',
 'B22-159',
 'B22-246',
 'B22-48',
 'B22-206',
 'B22-263',
 'B22-229',
 'B22-69',
 'B22-227',
 'B22-02',
 'B22-146',
 'B22-08',
 'B22-267',
 'B22-06',
 'B22-202',
 'B22-269',
 'B22-169',
 'B22-242',
 'B22-208',
 'B22-237',
 'B22-12',
 'B22-73',
 'B22-212',
 'B22-233',
 'B22-294',
 'B22-252',


In [ ]:
from PIL import Image
import pathlib
import pandas as pd
import torchvision
import numpy as np


def transform_image(image, size):
    return image.resize(size)

class HoldoutDataset():
    def __init__(self):
        path_to_holdout = r"HoldOut"
        patient_directories = [patient for patient in pathlib.Path(path_to_holdout).iterdir()]
        path_to_Patient_Diagnois = r'PatientDiagnosis.csv'
        patient_diagnosisDF = pd.read_csv(path_to_Patient_Diagnois)
        patient_diagnosisDF = patient_diagnosisDF[(patient_diagnosisDF['DENSITAT'] == 'ALTA') | (patient_diagnosisDF['DENSITAT'] == 'NEGATIVA')]
        patient_diagnosisDF['DENSITAT'] = [1 if x == 'ALTA' else -1 for x in patient_diagnosisDF['DENSITAT']]
        patient_diagnosisDF = patient_diagnosisDF.rename(columns={'DENSITAT': 'DiagnosisGT'})
        
        self.dictionary = {}
        
        for patient in patient_directories:
            if patient.name[:-2] in patient_diagnosisDF['CODI'].values:
                images = [x for x in patient.iterdir() if x.is_file() and x.name.endswith('.png')]
                self.dictionary[patient.name[:-2]] = (images, patient_diagnosisDF[patient_diagnosisDF['CODI'] == patient.name[:-2]]['DiagnosisGT'].values)
        
    def __len__(self):
        return len(self.dictionary)
    
    def __getitem__(self, idx):
        patient_id = list(self.dictionary.keys())[idx]
        images, patient_diagnosis = self.dictionary[patient_id]
        transformed_images = [transform_image(Image.open(image_path).convert("RGB"), (256, 256)) for image_path in images]
        return transformed_images, patient_id, patient_diagnosis


holdout_dataset = HoldoutDataset() 
print(len(holdout_dataset))

82


: 

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader
from tqdm import tqdm


# Define an Attention Mechanism
class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.attention_weights = nn.Linear(input_dim, 1, bias=False)

    def forward(self, features):
        """
        Args:
            features: Tensor of shape (num_patches, feature_dim)
        Returns:
            weighted_output: Tensor of shape (feature_dim,)
        """
        scores = self.attention_weights(features)  # Shape: (num_patches, 1)
        scores = torch.softmax(scores, dim=0)  # Normalize scores across patches
        weighted_output = torch.sum(features * scores, dim=0)  # Weighted sum of features
        return weighted_output, scores


# Initialize models
models = []
for n in range(5):
    model = HelicobacterClassifier()
    model.load_state_dict(torch.load(rf"HelicobacterClassifier_fold{n}.pth"))   
    models.append(model.to(device))

# Initialize Attention
attention = Attention(input_dim=2).to(device)  # Assuming the output features are probabilities with 2 classes

#--------------------------------------------------------------

# Create a DataLoader for the holdout dataset
holdout_loader = DataLoader(holdout_dataset, batch_size=1, shuffle=False, collate_fn=lambda x: x)

# Predict diagnosis for each patient using ensemble with attention
predictions = []
ground_truth = []

for data in tqdm(holdout_loader, desc="Processing patients"):
    data = data[0]  # Remove batch dimension
    images, patient_id, patient_diagnosis = data
    images = torch.stack([torchvision.transforms.ToTensor()(image) for image in images])
    images = images.to(device)
    
    # Extract outputs from all models for all patches
    patch_features = []
    for img in images:
        img = img.unsqueeze(0)  # Add batch dimension
        outputs = [torch.softmax(model(img), dim=1) for model in models]  # Shape: (num_models, num_classes)
        outputs = torch.mean(torch.stack(outputs), dim=0)  # Average over models
        patch_features.append(outputs.squeeze(0))  # Shape: (num_classes,)
    
    patch_features = torch.stack(patch_features)  # Shape: (num_patches, num_classes)
    
    # Apply attention mechanism
    weighted_output, scores = attention(patch_features)  # Shape: (num_classes,), (num_patches, 1)
    _, predicted = torch.max(weighted_output, dim=0)  # Final prediction based on weighted output
    
    # Interpret diagnosis
    final_prediction = 1 if predicted.item() == 1 else -1
    predictions.append(final_prediction)
    ground_truth.append(patient_diagnosis[0].item())

    print(f"Patient ID: {patient_id}, Predicted Diagnosis: {final_prediction}, Ground Truth: {patient_diagnosis[0].item()}")


/var/folders/7s/p9vc6dt51918j4q47v387x880000gn/T/ipykernel_48303/3515831914.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(rf"Helicoba

Patient ID: B22-88, Predicted Diagnosis: -1, Ground Truth: -1


In [ ]:
# Compute confusion matrix
conf_matrix = confusion_matrix(ground_truth, predictions)
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Compute classification report
class_report = classification_report(ground_truth, predictions, target_names=['Negative', 'Positive'])
print("Classification Report:")
print(class_report)